Pomysłem na rozwiązanie problemu rekomendacji jest stworzenie sieci neuronowej, która oceni filmy zgodnie z preferencjami danego użytkownika.

 W tym celu planuję przygotować dane, aby wektor wejściowy do sieci zawierał zarówno reprezentację ocenianego filmu, jak i reprezentację użytkownika. Posiadam jedynie dane dotyczące preferencji użytkowników, zatem będzie to jedyny wyznacznik.

In [42]:
import pandas as pd
movie=pd.read_csv(r"ml-25m\movies.csv")
rat=pd.read_csv(r"ml-25m\ratings.csv")
tag=pd.read_csv(r"ml-25m\tags.csv")
gs=pd.read_csv(r"ml-25m\genome-scores.csv")
gt=pd.read_csv(r"ml-25m\genome-tags.csv")

**Zaczniemy od reorganizacji danych z tabeli genome_scores.**

Dostaracza ona dużo informacji na temat treści filmu. Filmy są tam ocenione pod względem zawartości ponad 1000 tagów. Są podane jednak w oszczędnej dla pamięci formie, więc zreorganizuję tabele aby dostoswać ją pod wygodny wektor danych.
 

In [43]:

data = gs.pivot(index='movieId', columns='tagId', values='relevance')
data.columns.name = None
data.head()

,1,2,3,4,5,6,7,8,9,10,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02875,0.02375,0.06250,0.07575,0.14075,0.14675,0.06350,0.20375,0.2020,0.03075,...,0.04050,0.01425,0.03050,0.03500,0.14125,0.05775,0.03900,0.02975,0.08475,0.02200
2,0.04125,0.04050,0.06275,0.08275,0.09100,0.06125,0.06925,0.09600,0.0765,0.05250,...,0.05250,0.01575,0.01250,0.02000,0.12225,0.03275,0.02100,0.01100,0.10525,0.01975
3,0.04675,0.05550,0.02925,0.08700,0.04750,0.04775,0.04600,0.14275,0.0285,0.03875,...,0.06275,0.01950,0.02225,0.02300,0.12200,0.03475,0.01700,0.01800,0.09100,0.01775
4,0.03425,0.03800,0.04050,0.03100,0.06500,0.03575,0.02900,0.08650,0.0320,0.03150,...,0.05325,0.02800,0.01675,0.03875,0.18200,0.07050,0.01625,0.01425,0.08850,0.01500
5,0.04300,0.05325,0.03800,0.04100,0.05400,0.06725,0.02775,0.07650,0.0215,0.02975,...,0.05350,0.02050,0.01425,0.02550,0.19225,0.02675,0.01625,0.01300,0.08700,0.01600


**Kolejnym celem jest utworzenie reprezentacji uzytkownika**

Postanowiłem zrobić to poprzez wektor jego ulubionych filmów, jako ze każdy film jest okreslony przez ponad 1000 cech postanowilem zmiejszyć ich liczbę i uszeregowac je wzgledem ważności i wybrać tylko te najwazniejsze do wektora jego preferencji. Chciałem aby sieć przyjmowała na wejsciu wektor reprezentujący użytkownika złożony z pewnej liczby jego ulubionych filmów które będą reprezentowane przez x najważniejszych cech, oraz wektor filmu do oceny stworzony w podobny sposób

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
n_components = 600

pca = PCA(n_components=n_components)
pca.fit(data)

explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)
print(cumulative_explained_variance)
import matplotlib.pyplot as plt

plt.plot(range(1, n_components + 1), cumulative_explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.grid()
plt.show()


Jak widać wyzej przy 600 cechach, zachowujemy prawie 96% informacji,
jednak wystarczy jedynie 25 cech aby zachować około 60% informacji.
Postanowiłem że wektory wejściowe będą złożone z 50 cech filmu ocenianego i 25 filmów należących do ulubionych uzytkownika.

In [44]:
import numpy as np
from sklearn.decomposition import PCA
n_components = 50


pca = PCA(n_components=n_components)
pca.fit(data)

explained_variance = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance)
print(cumulative_explained_variance)


[0.14905495 0.22326949 0.28168725 0.32124008 0.35382191 0.38187087
 0.40660537 0.4287136  0.44887958 0.46572842 0.48076429 0.49415438
 0.50713443 0.51925706 0.53084176 0.54110816 0.55019388 0.5590928
 0.56708517 0.57490315 0.58210175 0.58900122 0.5957148  0.6020578
 0.60809961 0.61370185 0.61891792 0.62385818 0.62871808 0.63337181
 0.6379532  0.64222105 0.64622715 0.65003532 0.65363208 0.65719773
 0.66063974 0.66397997 0.66722317 0.67033736 0.67334321 0.67630123
 0.67919024 0.6820058  0.68475618 0.68749435 0.69012061 0.69262602
 0.69507328 0.69742407]


In [ ]:
for i, explained_variancex in enumerate(explained_variance):
    print(f"Komponent główny {i+1}: Wyjaśniona wariancja = {explained_variancex:.4f}, Kumulatywna wyjaśniona wariancja = {cumulative_explained_variance[i]:.4f}")

DataFrame reduced_df zawiera cechy z analizy PCA przypisane do id filmu.

In [45]:
reduced_features = pca.transform(data)
reduced_df = pd.DataFrame(reduced_features, index=data.index, columns=[f"PC{i}" for i in range(1, n_components + 1)])
print(reduced_df)

              PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
movieId                                                                         
1        0.796514  2.212839 -2.489444  1.624747  0.984763  1.168046 -0.203300   
2       -1.541922  1.597864 -1.113613  1.302526 -0.196336  0.191809 -0.026161   
3       -1.506092 -0.010220 -0.605981 -0.316252  0.330972  0.334079  0.048253   
4       -1.293604 -0.775662 -0.926582 -0.384600  0.103713 -0.345682  0.683775   
5       -1.694557 -0.185986 -1.204239 -0.149549  0.470713  0.053199  0.385026   
...           ...       ...       ...       ...       ...       ...       ...   
205072  -0.034354  1.189431 -0.941385  0.439289  1.349124  0.065643 -0.668149   
205076  -0.294929 -1.243930 -0.833182 -0.320838  0.265652  0.273416 -0.106014   
205383   0.569254  0.952059  0.161266 -0.542846  0.214419  0.119859 -0.036601   
205425   1.551432  1.517558  0.109042 -0.184095  1.932083  0.367802 -0.280007   
206499   0.212250  1.392884 

**Scalenie danych dotyczacacy ocen z tymi dotyczacymi treści.**

Postanowiłem usunąć dane dotyczące daty, ponieważ ta cecha wydaje się znancząca głównie w wypadku aktualnych danych, aby zauwazyć współczesne trendy, a dane które posiadam są juz dosyć stare. Jeśli chodzi o wykorzystanie sekwencji ogladania filmów, uznałem że nie jest to szczególnie ważny czynnik przy takim problemie.

In [ ]:
rat = rat.drop("timestamp", axis=1)
merged_table = rat.merge(reduced_df, on='movieId', how='left')
merged_table

Zauważyłem ze dużo uzytkowników nie ocenia "dobrze" nawet 10 filmów, dlatego postanowiłem ze mniej filmów będzie reprezentować ich preferencje.

In [ ]:

high_ratings = merged_table[merged_table['rating'] >= 3.5]
high_rating_counts = high_ratings.groupby('userId')['movieId'].count()
few_high_rating_users = high_rating_counts[high_rating_counts < 8].index
print(f"Użytkownicy z mniej niż 8 dobrze ocenionymi filmami: {few_high_rating_users}")


In [ ]:
print(high_rating_counts.info)
print(merged_table.info)

Użytkowników którzy nie ocenili nawet 8 filmów dobrze postanowiłem usunąć, nie stanowią oni znaczącej liczby na tle wszystkich użytkowników, ale ich preferencje nie byłyby dobrze reprezentowane - filmy które wciąż ocenili słabo byłyby uznane za te które im się podobały

In [ ]:
merged_table = merged_table[~merged_table['userId'].isin(few_high_rating_users)]


In [ ]:
merged_table.info

Teraz tworzę dataframe któy zawiera po 20 cech 8 ulubionych filmów użytkownika

In [ ]:
# Grupowanie danych według userId
grouped = merged_table.groupby('userId')
top_movies = pd.DataFrame()

# Iterowanie po każdej użytkowniku
for user_id, group in grouped:
    # Wybór 5 najwyżej ocenionych filmów
    top_8 = group.nlargest(8, 'rating')[[f'PC{i}' for i in range(1, 21)]]
    top_8['userId'] = user_id
    top_8.set_index('userId', inplace=True)
    top_movies = pd.concat([top_movies, top_8])
top_movies.to_csv('my_dataframe50.csv')

In [ ]:
top_movies = pd.read_csv("my_dataframe50.csv")

In [ ]:
n = 8 # number of rows to merge
arr = top_movies.to_numpy().reshape(-1, n * top_movies.shape[1])
new_df = pd.DataFrame(arr, index=top_movies.index.unique())
print(new_df)

In [ ]:
new_df.to_csv('my_dataframe50.csv')

In [ ]:
import pandas as pd
new_dfw = pd.read_csv("my_dataframe50.csv")

In [ ]:
print(new_dfw.info())

In [ ]:
new_dfw=new_dfw.astype('float32')
merged_table=merged_table.astype('float32')
result = pd.merge(merged_table, new_dfw, on='userId')

# Wyświetlenie wynikowego DataFrame
print(result)

In [ ]:

print(result.info())

print(result.isnull().any(axis=1).sum())


In [ ]:
del merged_table, new_dfw , rat, data,  gs,reduced_df, reduced_features,movie,tag,gt

In [ ]:
result=result.dropna()


In [ ]:
from sklearn.model_selection import train_test_split

y = result['rating']
X = result.drop(['userId', 'movieId', 'rating'], axis=1)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
del X,y,result

In [ ]:
vector_1, vector_2=X_train.iloc[:, :50], X_train.iloc[:, 50:]

In [ ]:
del X_train

Zdecydowałem się głownie na funkcje aktywacji Leaky Relu ze względu na to że sporo danych wejściowych to liczby ujemne, więc wolę uniknąć martwych neuronów.


In [ ]:
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.initializers import he_normal

from keras.layers import LeakyReLU
import numpy as np
from keras.layers import Activation, Lambda

# Definiujemy wejścia
input_1 = Input(shape=(160,))
input_2 = Input(shape=(50,))

hidden_1 = Dense(160,kernel_initializer=he_normal())(input_1)
hidden_1 = LeakyReLU(alpha=0.01)(hidden_1)
hidden_11 = Dense(50,kernel_initializer=he_normal())(hidden_1)
hidden_11 = LeakyReLU(alpha=0.01)(hidden_11)
hidden_2 = Dense(50,kernel_initializer=he_normal())(input_2)
hidden_2 = LeakyReLU(alpha=0.01)(hidden_2)

# Konkatenacja warstw i warstwy, które mają wykryć zależności między
concatenated = concatenate([hidden_11, hidden_2])
hidden_ac = Dense(100,kernel_initializer=he_normal())(concatenated)
hidden_ac = LeakyReLU(alpha=0.01)(hidden_ac)
hidden_ac1 = Dense(100,kernel_initializer=he_normal())(hidden_ac)
hidden_ac1 = LeakyReLU(alpha=0.01)(hidden_ac1)
hidden_ac2 = Dense(100,kernel_initializer=he_normal())(hidden_ac1)
hidden_ac2 = LeakyReLU(alpha=0.01)(hidden_ac2)
hidden_ac3 = Dense(100,kernel_initializer=he_normal())(hidden_ac2)
hidden_ac3 = LeakyReLU(alpha=0.01)(hidden_ac3)
# Warstwa wyjściowa
output = Dense(1)(hidden_ac3)
output = Activation('sigmoid')(output)
output = Lambda(lambda x: x * 5)(output)
# Tworzenie modelu
model = Model(inputs=[input_1, input_2], outputs=output)

# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
print(X_train.info(), y_train.info())

In [ ]:
print(X_train.head())

In [ ]:
vector_1.dropna()


In [ ]:
print(vector_1.isnull().any(axis=1).sum(), vector_2.isnull().any(axis=1).sum())

In [ ]:
print(y_train.isnull().sum())

In [ ]:
from tensorflow.keras.models import load_model
model1=load_model("recommendation_model1.h5")

In [ ]:
print(X_test.info)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,TensorBoard


model_checkpoint = ModelCheckpoint("recommendation_model2.h5", save_best_only=True)
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
tensorboard = TensorBoard(log_dir="logs")



model1.fit([vector_2,vector_1], y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[model_checkpoint, early_stopping, tensorboard])


In [ ]:
from tensorflow.keras.models import load_model
vector_1t = X_test.iloc[:, :50]
vector_2t = X_test.iloc[:, 50:]
model1=load_model("recommendation_model1.h5")

model1.evaluate([vector_2t,vector_1t], y_test)


In [ ]:
del X_test

In [ ]:
from tensorflow.keras.models import load_model
model1=load_model("recommendation_model1.h5")


In [ ]:
model1.summary()

In [ ]:


rows = reduced_df.loc[[168492,31148,192383,168492,31148,192383,168492,31148], ['PC' + str(i) for i in range(1, 21)]]
vector_usera=rows.stack().reset_index(drop=True)
input1pr = np.array(vector_usera).reshape(1, -1)

In [ ]:
vector_movie=reduced_df.loc[[858]]
input2pr = np.array(vector_movie).reshape(1, -1)
predictions = model.predict([input1pr,input2pr])
print(predictions)

In [ ]:
maxim=1
the_bests=1
ff={}
selected_indexes = reduced_df.index[(reduced_df.index >= 1) & (reduced_df.index < 209171)]
for i in selected_indexes:
    vector_movie=reduced_df.loc[[i]]
    input2pr = np.array(vector_movie).reshape(1, -1)
    predictions = model1.predict([input1pr,input2pr])
    print(i)
    print("bestie",the_bests, maxim)
    if predictions>4.0:
        ff[i]=predictions
    if predictions>maxim:
        the_bests=i
        maxim=predictions
       

In [ ]:

film_data = {}

for movie_id, prediction in ff.items():
    row_index = movie[movie['movieId'] == movie_id].index[0]
    
    film_title = movie.loc[row_index, 'title']
    
    film_data[film_title] = prediction[0][0]




In [ ]:
sorted_dict = dict(sorted(film_data.items(), key=lambda item: item[1], reverse=True))

print(sorted_dict)

In [ ]:
import sqlite3


conn = sqlite3.connect('movies.db')


columns = ', '.join(['PCA{} REAL'.format(i) for i in range(1, 51)])
conn.execute('CREATE TABLE movies (movieId INT, title TEXT, {})'.format(columns))


conn.close()

In [ ]:
column_mapping = {'PC{}'.format(i): 'PCA{}'.format(i) for i in range(1, 51)}
reduced_df = reduced_df.rename(columns=column_mapping)

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('movies.db')
columns = ['movieId'] + ['PCA{}'.format(i) for i in range(1, 51)]


reduced_df.to_sql('movies', conn, if_exists='append', index=True)

conn.close()



In [ ]:
movie=pd.read_csv(r"ml-25m\movies.csv")
movie = movie.loc[:, ['movieId', 'title']]

conn = sqlite3.connect('movies.db')
c = conn.cursor()


data = [(row['title'], row['movieId']) for _, row in movie.iterrows()]

c.executemany('UPDATE movies SET title = ? WHERE movieId = ?', data)


conn.commit()
conn.close()